## 条件过滤

1. 读取本地数据
2. 基本面因子（过滤指数成分停牌涨跌停）
3. 技术因子与事件驱动（可买可卖）

## 1_读取本地数据

In [1]:
from jaqs.data import DataView
from jaqs.data import RemoteDataService
import os
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
dataview_folder = '../JAQS_Data/hs300'
dv = DataView()
dv.load_dataview(dataview_folder)

Dataview loaded successfully.


## 2_基本面因子（过滤停牌涨跌停）
mask_fundamental要过滤的为True

In [3]:
def mask():

    df_index_member = dv.get_ts('index_member')
    mask_index_member = df_index_member == 0 #定义信号过滤条件-非指数成分
    
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'.encode('utf-8')
    dv.remove_field('limit_reached')
    dv.add_formula('limit_reached', 'Abs((open - Delay(close, 1)) / Delay(close, 1)) > 0.095', is_quarterly=False)
    df_limit_reached = dv.get_ts('limit_reached')
    mask_limit_reached = df_limit_reached ==1 #满足该条件的被ban掉(公式条件为True的)
    mask_all = np.logical_and(mask_limit_reached, np.logical_and(mask_sus, mask_index_member))
    return mask_all

In [4]:
mask_fundamental = mask()

In [5]:
dv.remove_field('mask_fundamental')
dv.append_df(mask_fundamental, 'mask_fundamental')

In [6]:
# dv.get_ts('mask_fundamental').shape

## 3_技术因子与事件驱动（可买可卖）
mask_index_member为要过滤的为True

can_enter与can_exit皆为可交易为True

In [ ]:
def mask_index():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = df_index_member==0 #定义信号过滤条件-非指数成分
    return mask_index_member

def limit_up_down():
    # 定义可买卖条件——未停牌、未涨跌停
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
    dv.remove_field('up_limit')
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False)
    # 跌停
    dv.remove_field('down_limit')
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

In [ ]:
mask_index_member = mask_index()
can_enter,can_exit = limit_up_down()

In [ ]:
dv.remove_field('mask_index_member')
dv.remove_field('can_enter')
dv.remove_field('can_exit')

In [ ]:
dv.append_df(mask_index_member, 'mask_index_member')
dv.append_df(can_enter, 'can_enter')
dv.append_df(can_exit, 'can_exit')

In [ ]:
print(dv.get_ts('can_enter').head())

In [ ]:
dv.save_dataview('../JAQS_Data/hs300')